In [ ]:
import numpy as np
import pandas as pd
from dotenv import load_dotenv
import os
from sklearn.neighbors import NearestNeighbors
from sklearn.manifold import TSNE
import spotipy
from sklearn.preprocessing import StandardScaler
from spotipy.oauth2 import SpotifyClientCredentials
import matplotlib.pyplot as plt
import plotly.express as px

import nltk
from nltk import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

import string
import re